<a href="https://colab.research.google.com/github/GANGADHAR99890/AI-Voice-Assistant/blob/main/voice-bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import base64
import time
import speech_recognition as sr
import sqlite3
from datetime import datetime
from gtts import gTTS
from IPython.display import Audio, display, Javascript
from google.colab import output


recorded_audio_data = None

def record_audio(duration=5, filename="recorded.wav"):
    """
    Records audio directly from the user's microphone for a given duration (in seconds)
    and saves it as a WAV file.
    """
    global recorded_audio_data
    recorded_audio_data = None


    def record_callback(result):
        global recorded_audio_data
        recorded_audio_data = result

    output.register_callback('notebook.recorded', record_callback)

    js_code = f"""
    async function recordAudio(duration) {{
      const sleep = ms => new Promise(resolve => setTimeout(resolve, ms));
      try {{
        // Request microphone access.
        const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
        const recorder = new MediaRecorder(stream);
        let data = [];
        recorder.ondataavailable = event => data.push(event.data);
        recorder.start();
        // Record for the specified duration.
        await sleep(duration * 1000);
        recorder.stop();
        await new Promise(resolve => recorder.onstop = resolve);
        // Create a Blob from the recorded data.
        let blob = new Blob(data, {{ type: 'audio/wav' }});
        let reader = new FileReader();
        reader.readAsDataURL(blob);
        await new Promise(resolve => reader.onloadend = resolve);
        // Pass the Base64 string back to Python.
        google.colab.kernel.invokeFunction('notebook.recorded', [reader.result], {{}});
      }} catch (error) {{
        google.colab.kernel.invokeFunction('notebook.recorded', ['ERROR:' + error.toString()], {{}});
      }}
    }}
    recordAudio({duration});
    """
    display(Javascript(js_code))


    timeout = time.time() + duration + 20
    while recorded_audio_data is None and time.time() < timeout:
        time.sleep(0.1)
    if recorded_audio_data is None:
        raise RuntimeError("Recording failed or timed out. Please ensure you granted microphone access.")

    if recorded_audio_data.startswith("ERROR:"):
        raise RuntimeError(f"Error during recording: {recorded_audio_data}")

    # Decode the Base64 string and save as a WAV file.
    binary = base64.b64decode(recorded_audio_data.split(',')[1])
    with open(filename, "wb") as f:
        f.write(binary)
    print(f"Audio recorded and saved as {filename}")
    return filename

def speech_to_text_from_mic():
    """
    Uses the record_audio() function to capture voice input, then transcribes it using Google Speech Recognition.
    """
    try:
        audio_filename = record_audio(duration=5, filename="recorded.wav")
    except Exception as e:
        print("Error during recording:", e)
        return None

    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio_filename) as source:
            audio = recognizer.record(source)
    except Exception as e:
        print("Error reading the audio file:", e)
        return None

    try:
        text = recognizer.recognize_google(audio)
        print("You said:", text)
        return text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio.")
        return None
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        return None

def analyze_intent(text):
    """
    A simple rule-based intent analyzer.
    """
    text_lower = text.lower()
    if "account" in text_lower:
        if "balance" in text_lower:
            return "account_balance"
        return "account_query"
    elif "faq" in text_lower or "question" in text_lower:
        return "faq_query"
    else:
        return "general_query"

def generate_response(intent, user_text):
    """
    Returns a pre-defined response based on the detected intent.
    """
    responses = {
        "account_query": "I can help with your account information. Could you please provide your account number or more details?",
        "account_balance": "Your account balance is $1,234.56.",
        "faq_query": "Here are some frequently asked questions: For billing issues, call 1-800-XXX-XXXX. For technical support, visit our support page.",
        "general_query": "I'm not sure I understand. Could you please clarify your question?"
    }
    return responses.get(intent, "I'm sorry, I didn't understand that.")

def speak_text(text):
    """
    Converts text to speech using gTTS and plays it within Colab.
    """
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    display(Audio("response.mp3", autoplay=True))

def log_interaction(user_text, intent, response):
    """
    Logs each interaction in a SQLite database.
    """
    conn = sqlite3.connect('voice_bot.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS interactions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_text TEXT,
            intent TEXT,
            response TEXT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    c.execute('''
        INSERT INTO interactions (user_text, intent, response)
        VALUES (?, ?, ?)
    ''', (user_text, intent, response))
    conn.commit()
    conn.close()

def print_analytics():
    """
    Displays a simple analytics dashboard based on logged interactions.
    """
    conn = sqlite3.connect('voice_bot.db')
    c = conn.cursor()
    c.execute("SELECT COUNT(*) FROM interactions")
    total_interactions = c.fetchone()[0]
    print("\n--- Analytics Dashboard ---")
    print(f"Total interactions: {total_interactions}")
    c.execute("SELECT intent, COUNT(*) FROM interactions GROUP BY intent")
    intent_stats = c.fetchall()
    for intent, count in intent_stats:
        print(f"Intent: {intent} --> Count: {count}")
    conn.close()

def main():
    print("Welcome to the Intelligent Voice Bot!")
    while True:

        user_text = speech_to_text_from_mic()
        if user_text is None:
            print("Let's try again...\n")
            continue


        intent = analyze_intent(user_text)
        print(f"Identified intent: {intent}")

        response = generate_response(intent, user_text)
        print("Bot:", response)


        speak_text(response)


        log_interaction(user_text, intent, response)

        cont = input("\nWould you like to ask another question? (y/n): ").strip().lower()
        if cont != 'y':
            break

    print_analytics()
    print("Thank you for using the Intelligent Voice Bot!")

if __name__ == "__main__":
    main()


Welcome to the Intelligent Voice Bot!


<IPython.core.display.Javascript object>